#Modules Importation and preparation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
WORK_DIR = "/content/drive/My Drive/Colab Notebooks/Writer_identification"
!pip uninstall -y opencv-python
!pip install -U opencv-contrib-python
% cd "/content/drive/My Drive/Colab Notebooks/Writer_identification"

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 55.6MB 68kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30
/content/drive/My Drive/Colab Notebooks/Writer_identification


In [31]:
import Clusterer, Global_feature_exractors, Local_features_extractor, Norms, Image, PCA_reduction, Distances
from Dataset_loader import load_dataset_all, load_dataset_IAM
import json, os
import numpy as np
from testing import get_test_sample, accuracy_optimised

In [5]:
training_object = "training_09Sep"

In [8]:
with open("./config_train.json") as config_file:
    configuration = json.load(config_file)["Sift_Bow"][training_object]

In [11]:
BASE_FOLDER = configuration["base_folder"]
if not os.path.exists(BASE_FOLDER):
  os.mkdir(BASE_FOLDER)

#Train Model

In [12]:
def new_cluster_centers(images, path_to_save, nb_clusters, max_no_improvement):
	hellinger_normalization = Norms.Norm.hellinger_normalization
	local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

	images_pre = [Image.Image(path_image, local_features_extractor, verbose=True) for path_image in images]

	clusterer = Clusterer.Clusterer.fit_new_trainig(images_pre, 
																		path_to_save, 
																		nb_clusters=nb_clusters, 
																		max_no_improvement=max_no_improvement)

In [32]:
_, images = load_dataset_all(WORK_DIR)

In [33]:
nb_images_treat = configuration["nb_images_treat"]
new_cluster_centers(images=images[:nb_images_treat],
                    path_to_save=BASE_FOLDER+configuration["path_cluster_centers"], 
                    nb_clusters=configuration["Number_clusters"], 
                    max_no_improvement=configuration["max_no_improvement"])

I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
I have prepared an image !
Init 1/3 with method: k-means++
Inertia for init 1/3: 72.015094
Init 2/3 with method: k-means++
Inertia for init 2/3: 69.832904
Init 3/3 with method: k-means++
Inertia for init 3/3: 71.037495
Minibatch iteration 1/106500: mean batch inertia: 0.247626, ewa inertia: 0.247626 
Minibatch iteration 2/106500: mean batch inertia: 0.278272, ewa inertia: 0.247683 
Minibatch iteration 3/106500: mean batch inertia: 0.246775, ewa inertia: 0.247682 
Minibatch iteration

#Test Model

In [30]:
import importlib, Dataset_loader
importlib.reload(Dataset_loader)

<module 'Dataset_loader' from '/content/drive/My Drive/Colab Notebooks/Writer_identification/Dataset_loader.py'>

In [34]:
def test_sift_descr_bow(images, writers, cluster_centers_path, accuracy_calculator):
  
  hellinger_normalization = Norms.Norm.hellinger_normalization
  local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

  clusters_centers = Clusterer.Clusterer.fit_ancient_data(cluster_centers_path)
  bow = Global_feature_exractors.BOW(clusters_centers)

  images_pre = [Image.Image(path_image, local_features_extractor, global_feature_extractor=bow) for path_image in images]

  chi2_distance = Distances.Distance.chi2_distance
  accuracy_value = accuracy_calculator(images_pre, writers, bow, chi2_distance)

  return accuracy_value

In [36]:
writers, images = load_dataset_all(WORK_DIR)
nb_images_treat = configuration["test_sample_size"]
X_test, y_test = get_test_sample(images, writers, nb_images_treat)

In [41]:
accuracy_value= test_sift_descr_bow(images=X_test, 
                                    writers=y_test, 
                                    cluster_centers_path=BASE_FOLDER+configuration["path_cluster_centers"],
                                    accuracy_calculator=accuracy_optimised)

print()
print("Accuracy value for", training_object," :",accuracy_value*100,"%")

ICDAR_0245  ==>  ICDAR_0100 ( 0245_3 , 0100_3 )
ICDAR_0100  ==>  ICDAR_0245 ( 0100_3 , 0245_3 )
ICDAR_0004  ==>  ICDAR_0108 ( 0004_4 , 0108_4 )
IAM_235  ==>  IAM_235 ( e04-052 , e04-043 )
ICDAR_0004  ==>  ICDAR_0108 ( 0004_3 , 0108_3 )
IAM_338  ==>  IAM_338 ( g06-031g , g06-050g )
IAM_338  ==>  IAM_338 ( g06-050g , g06-031g )
ICDAR_0104  ==>  ICDAR_0100 ( 0104_4 , 0100_4 )
ICDAR_0031  ==>  ICDAR_0104 ( 0031_3 , 0104_3 )
IAM_209  ==>  IAM_209 ( d07-089 , d07-096 )
IAM_235  ==>  IAM_209 ( e04-043 , d07-089 )
ICDAR_0104  ==>  IAM_338 ( 0104_3 , g06-050g )
ICDAR_0108  ==>  ICDAR_0245 ( 0108_3 , 0245_4 )
ICDAR_0100  ==>  ICDAR_0108 ( 0100_4 , 0108_4 )
IAM_209  ==>  IAM_209 ( d07-096 , d07-089 )
ICDAR_0108  ==>  ICDAR_0100 ( 0108_4 , 0100_4 )
ICDAR_0031  ==>  ICDAR_0004 ( 0031_4 , 0004_3 )
ICDAR_0245  ==>  IAM_334 ( 0245_4 , g06-018c )
IAM_334  ==>  ICDAR_0245 ( g06-018c , 0245_4 )
IAM_334  ==>  IAM_334 ( g06-050c , g06-018c )

Accuracy value for training_09Sep  : 30.0 %
